In [5]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [6]:
# Load the data
file_path = "/Users/sherahuang/Documents/Kaggle/AMP-PD-prediction/data/"
peptides_df = pd.read_csv(f'{file_path}train_peptides.csv')
proteins_df = pd.read_csv(f'{file_path}train_proteins.csv')
clinical_df = pd.read_csv(f'{file_path}train_clinical_data.csv')

In [8]:
pep_merged = peptides_df.merge(clinical_df, on=['visit_id', 'visit_month', 'patient_id'])
prot_merged = proteins_df.merge(clinical_df, on=['visit_id', 'visit_month', 'patient_id'])
full = peptides_df.merge(prot_merged, on=['visit_id', 'visit_month', 'patient_id'])

In [9]:
full

,visit_id,visit_month,patient_id,UniProt_x,Peptide,PeptideAbundance,UniProt_y,NPX,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication
0,55_0,0,55,O00391,NEQEQPLGQWHLS,11254.3,O00391,11254.3,10.0,6.0,15.0,NaN,NaN
1,55_0,0,55,O00391,NEQEQPLGQWHLS,11254.3,O00533,732430.0,10.0,6.0,15.0,NaN,NaN
2,55_0,0,55,O00391,NEQEQPLGQWHLS,11254.3,O00584,39585.8,10.0,6.0,15.0,NaN,NaN
3,55_0,0,55,O00391,NEQEQPLGQWHLS,11254.3,O14498,41526.9,10.0,6.0,15.0,NaN,NaN
4,55_0,0,55,O00391,NEQEQPLGQWHLS,11254.3,O14773,31238.0,10.0,6.0,15.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
197673061,58648_108,108,58648,Q9Y6R7,GATTSPGVYELSSR,12825.9,Q9UBX5,27387.8,6.0,0.0,0.0,NaN,NaN
197673062,58648_108,108,58648,Q9Y6R7,GATTSPGVYELSSR,12825.9,Q9UHG2,369437.0,6.0,0.0,0.0,NaN,NaN
197673063,58648_108,108,58648,Q9Y6R7,GATTSPGVYELSSR,12825.9,Q9UKV8,105830.0,6.0,0.0,0.0,NaN,NaN
197673064,58648_108,108,58648,Q9Y6R7,GATTSPGVYELSSR,12825.9,Q9Y646,21257.6,6.0,0.0,0.0,NaN,NaN


In [9]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Assuming you have loaded your dataset into a variable called 'data'

# Group the data by patient ID
full_grouped = full.groupby('patient_id')

# Find the maximum sequence length across all patients
max_sequence_length = full_grouped['visit_month'].count().max()

# Initialize empty lists to store the padded sequences and labels
padded_sequences = []
labels = []

# Iterate over each patient's data
for patient_id, group in full_grouped:
    # Convert the 'visit_month' column to a list
    visit_months = group['visit_month'].tolist()

    # Pad the time series to the maximum sequence length using zero-padding
    padded_sequence = pad_sequences([visit_months], maxlen=max_sequence_length, padding='post')
    
    # Append the padded sequence to the list
    padded_sequences.append(padded_sequence)
    
    # Assuming you have a target variable 'upd23b_clinical_state_on_medication'
    # Extract the labels for the patient and append them to the list
    labels.append(group['upd23b_clinical_state_on_medication'].values)
    
# Convert the lists to numpy arrays
padded_sequences = np.array(padded_sequences)
labels = np.array(labels)

# Shape of padded_sequences: (number_of_patients, max_sequence_length)
# Shape of labels: (number_of_patients,)

# Reshape the padded_sequences to match the input shape of LSTM
padded_sequences = np.expand_dims(padded_sequences, axis=-1)

# Now, you can use padded_sequences and labels as inputs to your LSTM RNN model
# Remember to split the data into train and test sets before training your model


2023-05-08 21:48:42.463677: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/var/folders/1m/qlgvd1tj4zj_xvwlsjfgf8g40000gn/T/ipykernel_22459/2877902172.py:33: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  labels = np.array(labels)


In [ ]:

import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Assuming you have the padded_sequences and labels from the previous code

# Set the maximum sequence length
max_sequence_length = 100

# Truncate or pad the sequences to the maximum length
padded_sequences = pad_sequences(padded_sequences, maxlen=max_sequence_length)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

# Create the LSTM model
model = Sequential()
model.add(LSTM(64, input_shape=(max_sequence_length, 1)))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


In [1]:
#to segment

import numpy as np

# Define the length of each segment/window
window_size = 100

# Generate a long sequence of data (example)
long_sequence = np.arange(1000)

# Calculate the total number of segments/windows
num_segments = len(long_sequence) // window_size

# Initialize an empty list to store the segments/windows
segments = []

# Segment the long sequence into smaller windows
for i in range(num_segments):
    start = i * window_size
    end = start + window_size
    segment = long_sequence[start:end]
    segments.append(segment)

# Convert the list of segments to a numpy array
segments = np.array(segments)

# Print the segments
for segment in segments:
    print(segment)


[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 98 99]
[100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117
 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135
 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153
 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171
 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189
 190 191 192 193 194 195 196 197 198 199]
[200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217
 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235
 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253
 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270

In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense

from tensorflow.keras.preprocessing.sequence import pad_sequences


# Assuming you have segmented sequences stored in 'segments' numpy array

# Pad the sequences to a fixed length
max_sequence_length = max(len(seq) for seq in segments)
padded_sequences = pad_sequences(segments, maxlen=max_sequence_length, padding='post')

# Generate labels for the segments (example)
labels = np.random.randint(0, 2, size=len(segments))

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

# Create the LSTM model
model = Sequential()
model.add(LSTM(64, input_shape=(max_sequence_length, 1)))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')


Epoch 1/10


2023-05-08 22:24:08.919589: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-08 22:24:08.922598: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-08 22:24:08.925176: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 [==============================] - ETA: 0s - loss: 0.6864 - accuracy: 0.5000

2023-05-08 22:24:11.332408: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-08 22:24:11.334499: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-08 22:24:11.337345: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 [==============================] - 3s 3s/step - loss: 0.6864 - accuracy: 0.5000 - val_loss: 0.6988 - val_accuracy: 0.5000
Epoch 2/10
1/1 [==============================] - 0s 53ms/step - loss: 0.6810 - accuracy: 0.5000 - val_loss: 0.6979 - val_accuracy: 0.5000
Epoch 3/10
1/1 [==============================] - 0s 50ms/step - loss: 0.6763 - accuracy: 0.5000 - val_loss: 0.6987 - val_accuracy: 0.5000
Epoch 4/10
1/1 [==============================] - 0s 49ms/step - loss: 0.6718 - accuracy: 0.6250 - val_loss: 0.6991 - val_accuracy: 0.0000e+00
Epoch 5/10
1/1 [==============================] - 0s 50ms/step - loss: 0.6674 - accuracy: 0.8750 - val_loss: 0.6967 - val_accuracy: 0.5000
Epoch 6/10
1/1 [==============================] - 0s 49ms/step - loss: 0.6634 - accuracy: 0.7500 - val_loss: 0.6899 - val_accuracy: 0.5000
Epoch 7/10
1/1 [==============================] - 0s 51ms/step - loss: 0.6606 - accuracy: 0.6250 - val_loss: 0.6812 - val_accuracy: 0.5000
Epoch 8/10
1/1 [====================

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import tensorflow as tf


# Group the data by patient ID
full_grouped = full.groupby('patient_id')

# Find the maximum sequence length across all patients
max_sequence_length = full_grouped['visit_month'].count().max()

# Initialize empty lists to store the padded sequences and labels
padded_sequences = []
labels = []

# Iterate over each patient's data
for patient_id, group in full_grouped:
    # Convert the 'visit_month' column to a list
    visit_months = group['visit_month'].tolist()

    # Pad the time series to the maximum sequence length using zero-padding
    padded_sequence = pad_sequences([visit_months], maxlen=max_sequence_length, padding='post')
    
    # Append the padded sequence to the list
    padded_sequences.append(padded_sequence)
    
    # Assuming you have a target variable 'upd23b_clinical_state_on_medication'
    # Extract the labels for the patient and append them to the list
    labels.append(group['upd23b_clinical_state_on_medication'].values)
    
# Convert the lists to numpy arrays
padded_sequences = np.array(padded_sequences)
labels = np.array(labels)

# Shape of padded_sequences: (number_of_patients, max_sequence_length)
# Shape of labels: (number_of_patients,)

# Reshape the padded_sequences to match the input shape of LSTM
padded_sequences = np.expand_dims(padded_sequences, axis=-1)

from sklearn.preprocessing import LabelEncoder

# Flatten the nested NumPy arrays in labels
labels_flat = [label for sublist in labels for label in sublist]

# Convert the flattened labels to numerical values using LabelEncoder
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels_flat)

# Convert the numerical labels to one-hot encoded format
encoder = OneHotEncoder(sparse=False)
labels_encoded = encoder.fit_transform(labels_encoded.reshape(-1, 1))

# Continue with the rest of the code for training and evaluation



# Convert NumPy arrays to TensorFlow tensors
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels_encoded, test_size=0.2, random_state=42)

X_train = tf.convert_to_tensor(X_train)
y_train = tf.convert_to_tensor(y_train)
X_test = tf.convert_to_tensor(X_test)
y_test = tf.convert_to_tensor(y_test)

# Create the LSTM model
model = Sequential()
model.add(LSTM(64, input_shape=(max_sequence_length, 1)))
model.add(Dense(labels_encoded.shape[1], activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test loss:", test_loss)
print("Test accuracy:", test_accuracy)



/var/folders/1m/qlgvd1tj4zj_xvwlsjfgf8g40000gn/T/ipykernel_23267/844774499.py:40: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  labels = np.array(labels)
